In [43]:
import pandas as pd

df = pd.read_csv('C:/D-drive-18921/College/AI System Design/ingredient-phrase-tagger/nyt-ingredients-snapshot-2015.csv')
df.shape

df.dropna(subset=['name', 'input'], inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.iloc[:, [0,1,2,3,4,5,6]]

garlic_ones = df[df['name'].str.contains('garlic clove|garlic cloves|cloves garlic| clove garlic')].index
garlic_ones = list(garlic_ones)
#print(garlic_ones)
#print(garlic_ones)
df.iloc[garlic_ones, 2] = 'garlic'
df.iloc[garlic_ones, 5] = 'clove'

cinnamon_ones = df[df['name'].str.contains('cinnamon stick|cinnamon sticks')].index
df.iloc[cinnamon_ones, 2] = 'cinnamon'
df.iloc[cinnamon_ones, 5] = 'stick'

df['unit'] = df['unit'].astype(str)
df = df[pd.to_numeric(df['qty'], errors='coerce').notnull()]
df['qty'] = df['qty'].astype(float)

df['input'] = df['input'].apply(lambda x: x.replace('tbsp', 'tablespoon'))
df['input'] = df['input'].apply(lambda x: x.replace('tsp', 'teaspoon'))
df['input'] = df['input'].apply(lambda x: x.replace('tbsps', 'tablespoon'))
df['input'] = df['input'].apply(lambda x: x.replace('tsps', 'teaspoon'))
df['input'] = df['input'].apply(lambda x: x.replace('ml ', ' millilitre '))
df['input'] = df['input'].apply(lambda x: x.replace('0g ', '0 gram '))
df['input'] = df['input'].apply(lambda x: x.replace('5g ', '5 gram '))

df['unit'] = df['unit'].apply(lambda x: x.replace('ml', 'millilitre'))
df['unit'] = df['unit'].apply(lambda x: x.replace('tbsp', 'tablespoon'))
df['unit'] = df['unit'].apply(lambda x: x.replace('tsp', 'teaspoon'))


# df['name'] = df['name'].apply(f)
# # df[df['name'].str.contains('garlic')]
# import numpy as np
# df.shape
# np.unique(df[df['name'].str.contains('garlic')]['name'])
# df['input'] = df["input"].replace({"tbsp": "tablespoon", "tsp": "teaspoon", "ml ": " millilitre "})
# df['unit'] = df["unit"].replace({"tbsp": "tablespoon", "tsp": "teaspoon", "ml": " millilitre "})

# df.columns
df.to_csv('nyt_filtered.csv', index=False)
# df[df['unit'].str.contains('millilitre')]

C:\Users\shreya\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
import pandas as pd
import numpy as np
import json
import subprocess

df = pd.read_csv('ingr.csv', encoding='latin-1')

df_new = df.groupby('recipe_id')['ingredients'].apply(list).reset_index(name='ingredients')
# df_new = df_new.iloc[0:5000, :]
ingredients = df_new['ingredients'].to_numpy()

parsed_ingr = []
i=0
for ingr in ingredients:
    with open('tmp/parser-input.txt', 'w') as f:
        f.write('\n'.join(str(ingr)))
    with open('tmp/parser-output.txt', 'w') as f:
        bash_cmd = "python bin/parse-ingredients.py tmp/parser-input.txt"
        process = subprocess.check_call(bash_cmd.split(), stdout=f)
    with open('tmp/parser-output.json', 'w') as f:
        bash_cmd = "python bin/convert-to-json.py tmp/parser-output.txt"
        process = subprocess.check_call(bash_cmd.split(), stdout=f)
    with open('tmp/parser-output.json', 'r') as f:
        pingr = json.load(f)
    parsed_ingr.append(pingr)
    if i%100==0:
        print("%d Iteration done")
    i+=1
        
np.savetxt('parsed_ingredients.csv', parsed_ingr, delimiter=',')

(5000, 2)